In [1]:
from PIL import Image #import Python Image Library
import numpy as np
import os
import glob
import re
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

2023-05-05 19:06:49.244022: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-05 19:06:49.497501: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-05 19:06:49.498461: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 19:06:50.514130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [62]:
x_train = []
y_train = []

x_test = []
y_test = []


for x in ["corn", "damaged", "no_corn"]:
    cnt = 0
    label = 0
    for pic in glob.glob(f"./classes/{x}/*.tif"):
        im = Image.open(pic)
        im = np.array(im)
        if (im.shape[0] == 100) and (im.shape[1] == 100):
            r = im[:, :, 0]
            g = im[:, :, 1]
            b = im[:, :, 2]
            if cnt < 10:
                x_test.append([r, g, b])
                y_test.append([label])
                cnt += 1
            else:
                x_train.append([r, g, b])
                y_train.append([label])
    label += 1

In [63]:
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

453
453
30
30


In [64]:
x_train = np.array(x_train).transpose(0, 2, 3, 1)
y_train = np.array(y_train)
x_test = np.array(x_test).transpose(0, 2, 3, 1)
y_test = np.array(y_test)

x_train.shape


(453, 100, 100, 3)

In [65]:
x_train = x_train.astype('float32')  # set x_train data type as float32
x_test = x_test.astype('float32')  # set x_test data type as float32
x_train /= 255  # change x_train value between 0 - 1
x_test /= 255  # change x_test value between 0 - 1
y_train = keras.utils.to_categorical(y_train, 3)  # change label to binary / categorical: [1 0 0 0] = 0, [0 1 0 0] = 1, so on
y_test = keras.utils.to_categorical(y_test, 3)



In [68]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[100, 100, 3]))
model.add(MaxPooling2D(pool_size=(2,2), padding='same')) #max pooling with stride (2,2)
model.add(Conv2D(32, (3, 3), activation='relu')) #layer convolutional 2D
model.add(MaxPooling2D(pool_size=(2,2))) #max pooling with stride (2,2)
model.add(Dropout(0.25)) #delete neuron randomly while training and remain 75%
model.add(Flatten()) #make layer flatten
model.add(Dense(128, activation='relu')) #fully connected layer
model.add(Dropout(0.5)) #delete neuron randomly and remain 50%
model.add(Dense(3, activation='softmax')) #softmax works

In [69]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])  # setting loss function and optimizer

model.fit(x_train, y_train,
          batch_size=20,
          epochs=50,
          verbose=1,
          validation_data=(x_test, y_test)) #training with epochs 100, batch size = 50

loss, acc = model.evaluate(x_test, y_test, verbose=0) #evaluate testing data and calculate loss and accuracy
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))


Epoch 1/50
23/23 [==============================] - 2s 70ms/step - loss: 0.0498 - accuracy: 0.9823 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/50
23/23 [==============================] - 1s 63ms/step - loss: 4.3420e-08 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/50
23/23 [==============================] - 1s 63ms/step - loss: 5.7894e-09 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/50
23/23 [==============================] - 1s 62ms/step - loss: 3.1579e-09 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/50
23/23 [==============================] - 1s 62ms/step - loss: 3.2105e-08 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/50
23/23 [==============================] - 1s 62ms/step - loss: 4.7368e-09 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/50
23/23 [==============================] - 1s 63ms/step - loss: 1.0526e-08 - accuracy: 1.0000 - val_l